In [1]:
from random import shuffle
import glob,h5py, cv2
import numpy as np
from numpy import dstack
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
from scipy.signal import resample
width = 128
height = 128
channels = 3
duration = 24.2
fps = 25
sub = 'All_subjects'
num_frames = duration*fps # axis = 1 dimension, also the # of frames are going to be padded to this value.
filename = 'Datasets/'+sub+'_spect_single.hdf5'
print('File to be created: ' + filename)

File to be created: Datasets/All_subjects_spect_single.hdf5


In [5]:
# load labels
trainy = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/*/labels/microDoppler/*.txt'
trainy_files = sorted(glob.glob(trainy))
print('Num. of label files: '+str(len(trainy_files)))
for i in range(0,5):
    print(str(i+1)+'. ' +trainy_files[i])

Num. of label files: 659
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/labels/microDoppler/11010014_1607632426_1.txt
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/labels/microDoppler/11010014_1607632426_2.txt
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/labels/microDoppler/11010014_1607632426_3.txt
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/labels/microDoppler/11010014_1607632426_4.txt
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/labels/microDoppler/11010014_1607632426_5.txt


In [7]:
# load images
trainpath = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/*/microDoppler/*.png'
train_files2 = sorted(glob.glob(trainpath))
len(train_files2)
train_files2[0]
train_files = []
for i in range(0,len(trainy_files)):
#     fname = trainy_files[i][-23:].replace("txt","") + "png"
    fname = trainy_files[i][-30:].replace("txt","png") # -23 for nonbinary labels
    fname = fname.replace('_binary','')
    idx = [j for j, s in enumerate(train_files2) if fname in s] #train_files2.index(fname) 
    if len(idx) == 1:
        train_files.append(train_files2[idx[0]])
    else:
        print('2 identical filenames at i=',str(i))
print('Num. of label files: '+str(len(train_files)))
for i in range(0,5):    
    print(str(i+1)+'. '+train_files[i])

Num. of label files: 659
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/microDoppler/11010014_1607632426_1.png
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/microDoppler/11010014_1607632426_2.png
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/microDoppler/11010014_1607632426_3.png
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/microDoppler/11010014_1607632426_4.png
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/microDoppler/11010014_1607632426_5.png


## Load images

In [9]:
cnt = 1
images = []
winsize = 0.2 # sec
totalwin = int(24.2/winsize)
for image in train_files:
    img = cv2.imread(image)
    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    clear_output(wait=True)
    print('Loading image '+str(cnt)+'/'+str(len(train_files))+ ' '+image)
    images.append(img)  
    cnt += 1
    
images = np.array(images)/255.
images.shape

Loading image 659/659 /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/15 jan emin/microDoppler/11050008_1610731475_1.png


(659, 128, 128, 3)

In [10]:
x_train, x_test = train_test_split(images, test_size=0.2, random_state=1)

In [12]:
f = h5py.File(filename, mode='w')
f.create_dataset("train_img", x_train.shape, np.uint8)
f.create_dataset("test_img", x_test.shape, np.uint8)  

f["train_img"][...] = x_train
f["test_img"][...] = x_test
f.close()
print(filename+' created.')

Datasets/All_subjects_spect_single.hdf5 created.
